# Customize gate calibrations

Quantum hardware researches frequently require knowledge of native gate implementations on a specific devices, in the form of pulse sequences. The pulse implementation of gates is called "gate calibrations". In Amazon Braket, you have access to native gate calibrations provided by quantum hardware providers. Gaining access to gate calibrations empowers researchers to experiment with quantum gate designs and execute quantum programs with customized, improved gate calibrations.

In [20]:
import numpy as np

import braket.experimental.autoqasm as aq
from braket.experimental.autoqasm import pulse
from braket.experimental.autoqasm.instructions import x, h, y, z, rx, rz, cz, measure

from braket.aws import AwsDevice
from braket.devices import Devices
from braket.circuits import Gate, QubitSet, FreeParameter
from braket.pulse import DragGaussianWaveform


# Rigetti Aspen-M-3 device is the targeted device for this example notebook
device = AwsDevice(Devices.Rigetti.AspenM3)

## Gate calibrations from hardware providers

Let's start by viewing the native gate calibrations supplied by hardware providers. 

In [9]:
provider_calibrations = device.gate_calibrations

Even of the same gate, different physical qubits and angles often have a different pulse implementation. So, each gate calibration is associated with certain physical qubits and angles. In the code snippet below, we retrieve the gate calibration of Rx gate for physical qubit `$0` and `angle=pi/2`.

In [10]:
pulse_sequence_rx_pi_2_q0 = provider_calibrations.pulse_sequences[(Gate.Rx(np.pi / 2), QubitSet(0))]

print(pulse_sequence_rx_pi_2_q0.to_ir())

OPENQASM 3.0;
cal {
    waveform wf_drag_gaussian_1 = drag_gaussian(24.0ns, 2.547965400864ns, 2.438480469475024e-10, 0.294131242904595, false);
    barrier $0;
    shift_frequency(q0_rf_frame, -321047.14178613486);
    play(q0_rf_frame, wf_drag_gaussian_1);
    shift_frequency(q0_rf_frame, 321047.14178613486);
    barrier $0;
}


Some gate calibrations are defined not with fixed value of angles, but rather with a variable. In this case, these gate calibrations can be retrieved with angles of `FreeParameter` type. For example, the code snippet below retrieves the gate calibration of XY gate on qubit pair `$0` and `$1`, and a variable angle `theta`.

In [18]:
theta = FreeParameter("theta")
pulse_sequence_xy_theta_q0_q1 = provider_calibrations.pulse_sequences[(Gate.XY(theta), QubitSet([0,1]))]

print(pulse_sequence_xy_theta_q0_q1.to_ir())

OPENQASM 3.0;
cal {
    waveform q0_q1_xy_sqrtiSWAP = {0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00017027814510117418 + 3.545814161029584e-05im, 0.0001637531384142647 - 0.0006963417291257475im, -0.0024236384535354085 - 0.0006359978840345775im, -0.0020880266459824227 + 0.0071991833817111215im, 0.0183154639363212 + 0.0058256555691840435im, 0.01389647133058614 - 0.0400970302233641im, -0.07601004972687501 - 0.028547907918616208im, -0.05096895434492601 + 0.12580109168386006im, 0.18377215049282655 + 0.08000540771043592im, 0.11203349617531491 - 0.24029683804396326im, -0.2861666410762052 - 0.1424698085519111im, -0.1679238652056395 + 0.31665740387208713im, 0.3324698576367831 + 0.18739403644354738im, 0.20188025689491965 - 0.33761393282493973im, -0.3365382754087318 - 0.21314702103059255im, -0.22270886691920647 + 0.3324455449040143im, 0.3270632885726885 + 0.2314788638652746im, 0.23987767413261177 - 0.32112639461245646im, -0.3148886746278293 - 0.2480571398104766im, -0.25605893770993265 + 0.30842396

## Compose custom gate calibrations with AutoQASM
To customize a gate calibration, you need to 
1. Find the definition of the gate.
2. Decide the qubits and angles you want to define the calibration.
3. Compose a pulse program that defines the calibration.

As an example, let's define the gate calibration of `rx` on qubit `$1` and at angle `pi/2`. We first inspect the definition of the `rx` gate, which takes qubit parameter `target` and angle parameter `angle`. A gate calibration for the `rx` gate must fully specify the inputs of the gate, `target` and `angle`.

In [28]:
help(rx)

Help on function rx in module braket.experimental.autoqasm.instructions.gates:

rx(target: Union[int, oqpy.classical_types._ClassicalVar, oqpy.base.OQPyExpression, str, oqpy.quantum_types.Qubit], angle: float) -> None
    X-axis rotation gate.
    
    Args:
        target (QubitIdentifierType): Target qubit.
        angle (float): Rotation angle in radians.



The pulse operations of the gate calibration is composed in the body of a function decorated by `@aq.gate_calibration`. To specify fixed values for the parameters `target` and `angle`, we set the values through keyword arguments of the decorator.

In [43]:
wf = DragGaussianWaveform(
    24e-9, 2.547965400864e-9, 2.370235498840002e-10, 0.293350447987059, False, "wf_drag_gaussian_1"
)
q0_rf_frame = device.frames['q0_rf_frame']

@aq.gate_calibration(implements=rx, target="$0", angle=np.pi/2)
def my_defcal():
    pulse.barrier("$0")
    pulse.shift_frequency(q0_rf_frame, -321047.14178613486)
    pulse.play(q0_rf_frame, wf)
    pulse.shift_frequency(q0_rf_frame, 321047.14178613486)
    pulse.barrier("$0")

After the gate calibration is defined, it is ready to be attach to a quantum program. Say, we want to attach `my_defcal` to the main quantum program `my_program`.

In [53]:
@aq.main
def my_program():
    rx("$0", np.pi/2)
    rz("$1", 0.123)
    measure("$0")

main_program = my_program()
print(main_program.to_ir())

OPENQASM 3.0;
rx(pi / 2) $0;
rz(0.123) $1;
bit __bit_0__;
__bit_0__ = measure $0;


We can use the `with_calibrations` method of the program which creates a new program with `my_defcal` attached, leaving the original intact. You can experiment with different gate calibrations on the same program without needing to redefine the main program every time.

In [54]:
custom_program = main_program.with_calibrations(my_defcal)
print(custom_program.to_ir())

OPENQASM 3.0;
extern drag_gaussian(duration, duration, float[64], float[64], bool) -> waveform;
waveform wf_drag_gaussian_1 = drag_gaussian(24.0ns, 2.547965400864ns, 2.370235498840002e-10, 0.293350447987059, false);
defcal rx(pi / 2) $0 {
    barrier $0;
    shift_frequency(q0_rf_frame, -321047.14178613486);
    play(q0_rf_frame, wf_drag_gaussian_1);
    shift_frequency(q0_rf_frame, 321047.14178613486);
    barrier $0;
}
rx(pi / 2) $0;
rz(0.123) $1;
bit __bit_0__;
__bit_0__ = measure $0;


You can also define a gate calibration with variable parameters. For variable parameters, instead of setting it in the decorator, you set it as a arguments of the decorated function. The parameters must have type hint. The type hint for qubits is `aq.Qubit`, and the type hint for angles is `float`. Let's define another gate calibration. This time, we define for `rz` gate on qubit `$1` and a variable angle `angle`.

In [55]:
@aq.gate_calibration(implements=rz, target="$1")
def my_defcal_2(angle: float):
    pulse.barrier("$1")
    pulse.shift_frequency(q0_rf_frame, -321047.14178613486)
    pulse.play(q0_rf_frame, wf)
    pulse.shift_frequency(q0_rf_frame, 321047.14178613486)
    pulse.delay(q0_rf_frame, angle*1e-8)
    pulse.barrier("$1")

We then attach this gate calibration, `my_defcal_2`, to the main program we composed previously, together with the other gate calibration, `my_defcal`.

In [57]:
custom_program = main_program.with_calibrations([my_defcal, my_defcal_2])
print(custom_program.to_ir())

OPENQASM 3.0;
extern drag_gaussian(duration, duration, float[64], float[64], bool) -> waveform;
waveform wf_drag_gaussian_1 = drag_gaussian(24.0ns, 2.547965400864ns, 2.370235498840002e-10, 0.293350447987059, false);
defcal rx(pi / 2) $0 {
    barrier $0;
    shift_frequency(q0_rf_frame, -321047.14178613486);
    play(q0_rf_frame, wf_drag_gaussian_1);
    shift_frequency(q0_rf_frame, 321047.14178613486);
    barrier $0;
}
defcal rz(angle[32] angle) $1 {
    barrier $1;
    shift_frequency(q0_rf_frame, -321047.14178613486);
    play(q0_rf_frame, wf_drag_gaussian_1);
    shift_frequency(q0_rf_frame, 321047.14178613486);
    delay[angle * 1e-08 * 1s] q0_rf_frame;
    barrier $1;
}
rx(pi / 2) $0;
rz(0.123) $1;
bit __bit_0__;
__bit_0__ = measure $0;


## Summary
This example notebook shows you how to retrieve and view the gate calibrations supplied by hardware providers. With AutoQASM, you can customize gate calibrations to explore different pulse implementation of gates. You can define gate calibrations against fixed values of gate parameters, as well as using parametric definition. Multiple gate calibration can be attached to a program to create a new program. This makes it easy to experiment with different gate calibrations on a same program.